In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import os
import state_code
from state_code import SAMPLE
import collections
import plotly.graph_objects as go
import numpy as np

In [2]:
source = requests.get('https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports_us/')

soup = bs(source.text,'html.parser')
page = soup.prettify()

In [4]:
web_fileset = set()
web_set = set()
dic_p = {}
for i in soup.find_all('a', class_='js-navigation-open'):
    if 'csv' in i['href']:
        web_set.add(i['href'])
        web_fileset.add(i['href'].split('/')[-1])
        dic_p[i['href'].split('/')[-1]] = i['href']

dir_set = set(os.listdir('./us_dataset'))
new_data = web_fileset.difference(dir_set)
for i in new_data:
    path = 'https://raw.githubusercontent.com'+''.join(dic_p[i].split('/blob'))
    # raw_source = requests.get(path)
    # raw_soup = bs(raw_source.text, 'html.parser')
    # csv_path = 'https://raw.githubusercontent.com'+raw_soup.find(id='raw-url')['href']
    print(path)
    df = pd.read_csv(path)
    filename = i.split('/')[-1]
    #print(filename)
    df.to_csv('./us_dataset/'+filename)


https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/04-15-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/04-25-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/04-16-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/04-13-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/04-20-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/04-17-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/04-21-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_c

In [8]:
files = os.listdir('./us_dataset/')
files.sort()

In [37]:

output_df = []
entire = pd.DataFrame(columns=['code','state','total_confirmed', 'death', 'recovered','testing_rate', 'hosp_rate', 'mortality_rate', 'date'])
for i in files:
    #print(i)
    total = {}
    raw_df = pd.read_csv('./us_dataset/'+i).fillna(0)

    country = 'Country_Region'
    state = 'Province_State'
    check = raw_df[country] == 'US'
    df = raw_df[check]

    for _, row in df.iterrows():
        us_state = row[state]
        if us_state not in state_code.CODE:
            #print(us_state)
            continue
        #print(row[state],i)
        if us_state in total:
            #print(total[temp])
            total[us_state][0] += row['Confirmed']
            total[us_state][1] += row['Deaths']
            total[us_state][2] += row['Recovered']
            total[us_state][3] += row['Testing_Rate']
            total[us_state][3] += row['Hospitalization_Rate']
            total[us_state][3] += row['Mortality_Rate']
        else:
            total[us_state] = [row['Confirmed'],row['Deaths'], row['Recovered'], row['Testing_Rate'], row['Hospitalization_Rate'], row['Mortality_Rate']]
            
    usa_df = pd.DataFrame(columns=['code','state','total_confirmed', 'death', 'recovered','testing_rate', 'hosp_rate', 'mortality_rate', 'date'])


    for j in total:
        a_row = [state_code.CODE[j], j]+total[j] + [i[:10]]
        leng = len(usa_df)
        usa_df.loc[leng] = a_row
        entire.loc[leng] = a_row
        #row_df = pd.DataFrame([a_row])
        # usa_df = usa_df.append(a_row, ignore_index=True)
    #print(usa_df)
    output_df.append(usa_df.fillna(0))

In [24]:
fig = go.Figure()

colors = ['#FFE9E5', '#FFAFA1', '#FF745C' , '#FF745C', '#9A1903']

# Add traces, one for each slider step
for i in output_df:
    #print(i['total_confirmed'].astype(int))
    fig.add_trace(
        go.Choropleth(
    locations=i['code'], # Spatial coordinates
    z = i['hosp_rate'], # Data to be color-coded,
    colorscale = 'reds',
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorbar_title = "People",
    hovertext = i['state'] + '<br>',
    ))

# Make 10th trace visible

last = len(output_df) - 1

fig.data[last].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="restyle",
        args=["visible", [False] * len(fig.data)],
        label= output_df[i]['date'][0][:5])
    step["args"][1][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=last,
    currentvalue={"prefix": "Date: "},
    pad={"t": 1},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    title_text = 'COVID-19 USA Data Visualization',
    geo_scope='usa', # limite map scope to USA
)


fig.show()

In [50]:
import plotly.express as px
#gapminder = px.data.gapminder()
gapminder = entire
px.choropleth(entire, scope='usa', locations="code", color="total_confirmed", hover_name="total_confirmed", animation_frame='date', animation_group="state",
           log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90])

TypeError: choropleth() got an unexpected keyword argument 'log_x'

In [55]:
entire

,code,state,total_confirmed,death,recovered,testing_rate,hosp_rate,mortality_rate,date
0,AL,Alabama,6026,209,0.0,1521.551870,13.923000,3.468304,04-25-2020
1,AK,Alaska,339,9,217.0,2631.873914,9.439528,2.654867,04-25-2020
2,AS,American Samoa,0,0,0.0,5.391708,0.000000,0.000000,04-25-2020
3,AZ,Arizona,6286,273,1345.0,858.777721,16.258352,4.342984,04-25-2020
4,AR,Arkansas,2911,48,964.0,1469.779960,9.996565,1.648918,04-25-2020
5,CA,California,42368,1689,0.0,1290.621807,11.440238,3.986499,04-25-2020
6,CO,Colorado,12968,684,2086.0,1002.132945,18.244911,5.274522,04-25-2020
7,CT,Connecticut,24583,1865,0.0,2176.598967,7.362812,7.586544,04-25-2020
8,DE,Delaware,3576,112,809.0,1817.483497,8.389262,3.131991,04-25-2020
9,FL,Florida,30839,1055,0.0,1568.808841,16.359156,3.420993,04-25-2020


In [ ]:
date = [i['date'][0] for i in output_df]

# make figure
fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

# fill in most of layout
fig_dict["layout"]["xaxis"] = {"range": [30, 85], "title": "Life Expectancy"}
fig_dict["layout"]["yaxis"] = {"title": "GDP per Capita", "type": "log"}
fig_dict["layout"]["hovermode"] = "closest"
fig_dict["layout"]["sliders"] = {
    "args": [
        "transition", {
            "duration": 400,
            "easing": "cubic-in-out"
        }
    ],
    "initialValue": "04-12-2020",
    "plotlycommand": "animate",
    "values": date,
    "visible": True
}
fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Year:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

# make data
year = 1952
for continent in continents:
    dataset_by_year = dataset[dataset["year"] == year]
    dataset_by_year_and_cont = dataset_by_year[
        dataset_by_year["continent"] == continent]

    data_dict = {
        "x": list(dataset_by_year_and_cont["lifeExp"]),
        "y": list(dataset_by_year_and_cont["gdpPercap"]),
        "mode": "markers",
        "text": list(dataset_by_year_and_cont["country"]),
        "marker": {
            "sizemode": "area",
            "sizeref": 200000,
            "size": list(dataset_by_year_and_cont["pop"])
        },
        "name": continent
    }
    fig_dict["data"].append(data_dict)

# make frames
for year in years:
    frame = {"data": [], "name": str(year)}
    for continent in continents:
        dataset_by_year = dataset[dataset["year"] == int(year)]
        dataset_by_year_and_cont = dataset_by_year[
            dataset_by_year["continent"] == continent]

        data_dict = {
            "x": list(dataset_by_year_and_cont["lifeExp"]),
            "y": list(dataset_by_year_and_cont["gdpPercap"]),
            "mode": "markers",
            "text": list(dataset_by_year_and_cont["country"]),
            "marker": {
                "sizemode": "area",
                "sizeref": 200000,
                "size": list(dataset_by_year_and_cont["pop"])
            },
            "name": continent
        }
        frame["data"].append(data_dict)

    fig_dict["frames"].append(frame)
    slider_step = {"args": [
        [year],
        {"frame": {"duration": 300, "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 300}}
    ],
        "label": year,
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)


fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)

fig.show()